# Automated Web Scrape Practice

In [9]:
# Imports
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

In [10]:
# Set up Splinter
browser = Browser('chrome')

In [11]:
# Visit the Quotes to Scrape site
url = 'https://kansascity.craigslist.org/search/syp#search=1~gallery~0~0'
browser.visit(url)

In [12]:
# Parse the HTML
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# Scrape the heading
list_names = soup.find_all('a', class_='cl-app-anchor text-only posting-title')
card_infos = soup.find_all('div', class_='gallery-card')

meta_soup = BeautifulSoup(str(card_infos), 'html.parser')
meta_infos = meta_soup.find_all('div', class_='meta')
price_infos = meta_soup.find_all('span', class_='priceinfo')

posting_time = []
posting_title = []
posting_price = []
posting_city = []
posting_state = []
posting_combined_location = []
search_type = []


for minfo in meta_infos:
    # Get Posting Time
    posting_time.append(minfo.text.split('·', 1)[0])
    
    # Get Posting Location
    posting_location = minfo.text.split('·', 1)[1]

    # Posting City and State Combined
    posting_combined_location.append(posting_location)

    # Split Posting City and State
    posting_city.append(posting_location.split(',', 1)[0])
    if len(posting_location.split(',', 1)) > 1:
        posting_state.append(posting_location.split(',', 1)[1])
    else:
        posting_state.append('')
    

for list_name in list_names:
    search_type.append('Computer Parts')
    posting_title.append(list_name.text)


for price_info in card_infos:
    # Strip '$' in case we want to use the value only
    splt_price = price_info.find('span', class_='priceinfo')
    if '$' in price_info.text:
        splt_price = price_info.text.split('$', 1)[1]

        # Added '$' back in (can be removed if needed)
        splt_price = '$' + splt_price.split('hide', 1)[0]

    posting_price.append(splt_price)


# Create the DataFrame
list_info_df = pd.DataFrame({'List_Type':search_type, 'Price': posting_price, 'Item_Title': posting_title, 'Time_Listed': posting_time, 'Location':posting_combined_location})
list_info_df.head(10)

,List_Type,Price,Item_Title,Time_Listed,Location
0,Computer Parts,$20,"Bundle 19 Cables and Cords (HDMI, USB, VGA, Pr...",22 mins ago,"Raymore, MO"
1,Computer Parts,$225,Asus Tuf Gaming RTX 3060 ti OC,5h ago,Pleasant Hill
2,Computer Parts,$10,"24"" Samsung LED Monitor S24D300",8h ago,Overland Park
3,Computer Parts,$60,Zeos Meridian 400 series Laptop Charger or AC ...,6/6,Overland Park
4,Computer Parts,$330,TrippLite battery backup 1000 1500 Computer ne...,6/6,Blue Springs
5,Computer Parts,$250,ECR4Kids 30-Bay Locking Laptop Or Tablet Charg...,6/6,Blue Springs
6,Computer Parts,$155,16 Bay Laptop Tablet Charging Cart Chromebook...,6/6,Blue Springs
7,Computer Parts,$35,HP 64xl ink cartridges - 2 pk new,6/5,Western Shawnee
8,Computer Parts,$20,OGIO Element Black Laptop Bag Case Messenger B...,6/5,mission
9,Computer Parts,$1,Bulk Cooling Fans NIDEC TA350DC Double Ball In...,6/5,"Kansas City, MO"


In [6]:
# Create CSV file to Output folder
list_info_df.to_csv('Output/kansascity_craigslist_computer_parts.csv',index=False)

In [7]:
# End the automated browsing section
browser.quit()